In [123]:
import pandas as pd
import numpy as np
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)

In [124]:
# 데이터 불러오기 
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [125]:
train.columns

Index(['index', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10',
       'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20',
       'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'introelapse',
       'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [126]:
# 기본 전처리 
train = train.drop('index', axis=1)
test = test.drop('index', axis=1)

In [127]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [128]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            35333 non-null  float64
 1   Q2            35305 non-null  float64
 2   Q3            35322 non-null  float64
 3   Q4            35327 non-null  float64
 4   Q5            35362 non-null  float64
 5   Q6            35320 non-null  float64
 6   Q7            35310 non-null  float64
 7   Q8            35344 non-null  float64
 8   Q9            35348 non-null  float64
 9   Q10           35232 non-null  float64
 10  Q11           35333 non-null  float64
 11  Q12           35303 non-null  float64
 12  Q13           35356 non-null  float64
 13  Q14           35350 non-null  float64
 14  Q15           35345 non-null  float64
 15  Q16           35367 non-null  float64
 16  Q17           35373 non-null  float64
 17  Q18           35305 non-null  float64
 18  Q19           35355 non-nu

# lapase 관련 변수는 skew가 심하므로 로그변환 처리

In [129]:
# 로그 변환
train.introelapse = np.log1p(train.introelapse)
train.testelapse = np.log1p(train.testelapse)
train.surveyelapse = np.log1p(train.surveyelapse)

test.introelapse = np.log1p(test.introelapse)
test.testelapse = np.log1p(test.testelapse)
test.surveyelapse = np.log1p(test.surveyelapse)

# question 답변 안함과 nerdiness는 크게 상관이 없어보임

## 결론: Tipi를 답변하지 않을 수록 nerd가 아닐 확률이 높다...?

# << missing 처리 기준 >>
# question은 최빈값으로 채우기
# TIPI는 0으로 새로운 응답변수 생성

In [130]:
Q1 = train['Q1'].mode()
Q2 = train['Q2'].mode()
Q3 = train['Q3'].mode()
Q4 = train['Q4'].mode()
Q5 = train['Q5'].mode()
Q6 = train['Q6'].mode()
Q7 = train['Q7'].mode()
Q8 = train['Q8'].mode()
Q9 = train['Q9'].mode()
Q10 = train['Q10'].mode()
Q11 = train['Q11'].mode()
Q12 = train['Q12'].mode()
Q13 = train['Q13'].mode()
Q14 = train['Q14'].mode()
Q15 = train['Q15'].mode()
Q16 = train['Q16'].mode()
Q17 = train['Q17'].mode()
Q18 = train['Q18'].mode()
Q19 = train['Q19'].mode()
Q20 = train['Q20'].mode()
Q21 = train['Q21'].mode()
Q22 = train['Q22'].mode()
Q23 = train['Q23'].mode()
Q24 = train['Q24'].mode()
Q25 = train['Q25'].mode()
Q26 = train['Q26'].mode()

In [131]:
train['Q1'] = train['Q1'].fillna(Q1[0])
train['Q2'] = train['Q2'].fillna(Q2[0])
train['Q3'] = train['Q3'].fillna(Q3[0])
train['Q4'] = train['Q4'].fillna(Q4[0])
train['Q5'] = train['Q5'].fillna(Q5[0])
train['Q6'] = train['Q6'].fillna(Q6[0])
train['Q7'] = train['Q7'].fillna(Q7[0])
train['Q8'] = train['Q8'].fillna(Q8[0])
train['Q9'] = train['Q9'].fillna(Q9[0])
train['Q10'] = train['Q10'].fillna(Q10[0])
train['Q11'] = train['Q11'].fillna(Q11[0])
train['Q12'] = train['Q12'].fillna(Q12[0])
train['Q13'] = train['Q13'].fillna(Q13[0])
train['Q14'] = train['Q14'].fillna(Q14[0])
train['Q15'] = train['Q15'].fillna(Q15[0])
train['Q16'] = train['Q16'].fillna(Q16[0])
train['Q17'] = train['Q17'].fillna(Q17[0])
train['Q18'] = train['Q18'].fillna(Q18[0])
train['Q19'] = train['Q19'].fillna(Q19[0])
train['Q20'] = train['Q20'].fillna(Q20[0])
train['Q21'] = train['Q21'].fillna(Q21[0])
train['Q22'] = train['Q22'].fillna(Q22[0])
train['Q23'] = train['Q23'].fillna(Q23[0])
train['Q24'] = train['Q24'].fillna(Q24[0])
train['Q25'] = train['Q25'].fillna(Q25[0])
train['Q26'] = train['Q26'].fillna(Q26[0])

test['Q1'] = test['Q1'].fillna(Q1[0])
test['Q2'] = test['Q2'].fillna(Q2[0])
test['Q3'] = test['Q3'].fillna(Q3[0])
test['Q4'] = test['Q4'].fillna(Q4[0])
test['Q5'] = test['Q5'].fillna(Q5[0])
test['Q6'] = test['Q6'].fillna(Q6[0])
test['Q7'] = test['Q7'].fillna(Q7[0])
test['Q8'] = test['Q8'].fillna(Q8[0])
test['Q9'] = test['Q9'].fillna(Q9[0])
test['Q10'] = test['Q10'].fillna(Q10[0])
test['Q11'] = test['Q11'].fillna(Q11[0])
test['Q12'] = test['Q12'].fillna(Q12[0])
test['Q13'] = test['Q13'].fillna(Q13[0])
test['Q14'] = test['Q14'].fillna(Q14[0])
test['Q15'] = test['Q15'].fillna(Q15[0])
test['Q16'] = test['Q16'].fillna(Q16[0])
test['Q17'] = test['Q17'].fillna(Q17[0])
test['Q18'] = test['Q18'].fillna(Q18[0])
test['Q19'] = test['Q19'].fillna(Q19[0])
test['Q20'] = test['Q20'].fillna(Q20[0])
test['Q21'] = test['Q21'].fillna(Q21[0])
test['Q22'] = test['Q22'].fillna(Q22[0])
test['Q23'] = test['Q23'].fillna(Q23[0])
test['Q24'] = test['Q24'].fillna(Q24[0])
test['Q25'] = test['Q25'].fillna(Q25[0])
test['Q26'] = test['Q26'].fillna(Q26[0])

In [132]:
# 0으로 채워줌 (새로운 변수 처럼 작동하도록)
tipi  = ['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10']
train[tipi] = train[tipi].fillna(0)
test[tipi] = test[tipi].fillna(0)

# 나이, 가족수 이상치 처리

#### 나이 답변을 이상하게 한 사람은은 nerd일까?

크게 상관없는듯 -> 결측치, 이상치(2999?)는 평균으로 채우기로 결정

In [133]:
def age_outlier(x):
    if x > 100:
        return round(train.age.mean())
    else: 
        return x

In [134]:
train.age = train.age.apply(age_outlier)
test.age = test.age.apply(age_outlier)

In [135]:
def age_grouping(x):
    if x <= 10:
        return 10
    elif x <= 20:
        return 20
    elif x <= 30:
        return 30
    elif x <= 40:
        return 40
    elif x <= 50:
        return 50
    elif x <= 60:
        return 60
    elif x <= 70:
        return 70
    elif x <= 80:
        return 80
    elif x <= 90:
        return 90
    elif x <= 100:
        return 100
    else: 
        return x

In [136]:
train.age = train.age.apply(age_grouping)
test.age = test.age.apply(age_grouping)

In [137]:
# familysize

In [138]:
train.familysize = train.familysize.apply(age_outlier)
test.familysize = test.familysize.apply(age_outlier)

In [139]:
train['familysize'] = train['familysize'].fillna(round(train.familysize.mean()))
test['familysize'] = test['familysize'].fillna(round(train.familysize.mean()))

# 신상정보를 답변하지 않은 것과 너드는 크게 상관이 없어보임 -> 최빈값으로 대체 처리 

# 신상정보(?) 변수 전처리

신상정보는 최빈값으로 넣음

In [140]:
edu = train['education'].mode()
gen = train['gender'].mode()
eng = train['engnat'].mode() 
han = train['hand'].mode()
ori = train['orientation'].mode()
vot = train['voted'].mode()
mar = train['married'].mode()
asd = train['ASD'].mode()
rel = train['religion'].mode()

In [141]:
train['education'] = train['education'].fillna(edu[0])
train['gender'] = train['gender'].fillna(gen[0])
train['engnat'] = train['engnat'].fillna(eng[0])
train['hand'] = train['hand'].fillna(han[0])
train['orientation'] = train['orientation'].fillna(ori[0])
train['voted'] = train['voted'].fillna(vot[0])
train['married'] = train['married'].fillna(mar[0])
train['ASD'] = train['ASD'].fillna(asd[0])
train['religion'] = train['religion'].fillna(rel[0])

test['education'] = test['education'].fillna(edu[0])
test['gender'] = test['gender'].fillna(gen[0])
test['engnat'] = test['engnat'].fillna(eng[0])
test['hand'] = test['hand'].fillna(han[0])
test['orientation'] = test['orientation'].fillna(ori[0])
test['voted'] = test['voted'].fillna(vot[0])
test['married'] = test['married'].fillna(mar[0])
test['ASD'] = test['ASD'].fillna(asd[0])
test['religion'] = test['religion'].fillna(rel[0])

In [142]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            15000 non-null  float64
 1   Q2            15000 non-null  float64
 2   Q3            15000 non-null  float64
 3   Q4            15000 non-null  float64
 4   Q5            15000 non-null  float64
 5   Q6            15000 non-null  float64
 6   Q7            15000 non-null  float64
 7   Q8            15000 non-null  float64
 8   Q9            15000 non-null  float64
 9   Q10           15000 non-null  float64
 10  Q11           15000 non-null  float64
 11  Q12           15000 non-null  float64
 12  Q13           15000 non-null  float64
 13  Q14           15000 non-null  float64
 14  Q15           15000 non-null  float64
 15  Q16           15000 non-null  float64
 16  Q17           15000 non-null  float64
 17  Q18           15000 non-null  float64
 18  Q19           15000 non-nu

# nerdiness랑 시각화

# 나라별로 조금씩 편차가 있는 것을 확인

# country

In [143]:
train.country.value_counts()

USA    7419
GBR    1109
CAN     915
AUS     525
DEU     473
PHL     261
BRA     240
IND     233
POL     210
FRA     208
ITA     186
NLD     175
SWE     152
MEX     136
FIN     133
NZL     124
MYS     121
IDN     110
NOR     106
SGP     102
ESP      94
DNK      91
ROU      87
IRL      79
ARG      74
PRT      73
TUR      70
RUS      63
CZE      62
HUN      56
BEL      55
GRC      54
ZAF      52
CHL      49
AUT      48
CHE      47
ISR      44
SRB      38
HRV      35
JPN      32
HKG      31
THA      30
BGR      30
ARE      28
PAK      27
UKR      22
SVN      21
KOR      21
LTU      20
LVA      20
SVK      19
VEN      17
COL      16
PRI      15
KEN      15
PER      15
URY      14
CRI      14
SAU      12
EST      12
VNM      12
IRN      11
BIH      11
TWN      11
EGY      10
JOR      10
HND       9
KWT       9
ALB       8
NGA       8
CHN       8
ISL       8
QAT       8
JAM       7
TTO       7
DOM       6
MKD       5
BGD       5
BHS       4
BHR       4
GEO       4
GHA       4
GUM       4
ECU 

In [144]:
# 100이하면 '기타'로처리
country_list = train.country.value_counts()[:40]
def func(country):
    if country in country_list:
        return country
    else:
        return '기타'

train['country'] = train['country'].apply(lambda x : func(x))
test['country'] = test['country'].apply(lambda x : func(x))

In [145]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            15000 non-null  float64
 1   Q2            15000 non-null  float64
 2   Q3            15000 non-null  float64
 3   Q4            15000 non-null  float64
 4   Q5            15000 non-null  float64
 5   Q6            15000 non-null  float64
 6   Q7            15000 non-null  float64
 7   Q8            15000 non-null  float64
 8   Q9            15000 non-null  float64
 9   Q10           15000 non-null  float64
 10  Q11           15000 non-null  float64
 11  Q12           15000 non-null  float64
 12  Q13           15000 non-null  float64
 13  Q14           15000 non-null  float64
 14  Q15           15000 non-null  float64
 15  Q16           15000 non-null  float64
 16  Q17           15000 non-null  float64
 17  Q18           15000 non-null  float64
 18  Q19           15000 non-nu

In [146]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            35452 non-null  float64
 1   Q2            35452 non-null  float64
 2   Q3            35452 non-null  float64
 3   Q4            35452 non-null  float64
 4   Q5            35452 non-null  float64
 5   Q6            35452 non-null  float64
 6   Q7            35452 non-null  float64
 7   Q8            35452 non-null  float64
 8   Q9            35452 non-null  float64
 9   Q10           35452 non-null  float64
 10  Q11           35452 non-null  float64
 11  Q12           35452 non-null  float64
 12  Q13           35452 non-null  float64
 13  Q14           35452 non-null  float64
 14  Q15           35452 non-null  float64
 15  Q16           35452 non-null  float64
 16  Q17           35452 non-null  float64
 17  Q18           35452 non-null  float64
 18  Q19           35452 non-nu

In [147]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'introelapse',
       'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [148]:
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,1.386294,6.317165,1.945910,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,1.791759,4.454347,4.795791,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,50,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,2.302585,4.691348,4.615121,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,50,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,1.098612,4.804021,4.941642,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,20,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,1.386294,6.463029,5.379897,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,20,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,2.564949,4.644391,5.087596,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,3.465736,4.672829,5.192957,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,50,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,2.890372,4.644391,5.129899,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,2.708050,4.234107,4.700480,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,30,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [149]:
train.engnat.value_counts()

1.0    9826
2.0    5174
Name: engnat, dtype: int64

In [150]:
# engnat / voted / ASD 
train['engnat'] = train['engnat'] - 1
train['voted'] = train['voted'] - 1
train['ASD'] = train['ASD'] - 1

test['engnat'] = test['engnat'] - 1
test['voted'] = test['voted'] - 1
test['ASD'] = test['ASD'] - 1

In [151]:
train.engnat.value_counts()

0.0    9826
1.0    5174
Name: engnat, dtype: int64

In [152]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'introelapse',
       'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [153]:
col = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'education',
       'urban', 'gender', 'age', 'hand', 'religion', 'orientation',
    'married']

In [154]:
train = pd.get_dummies(columns = col , data = train)
test = pd.get_dummies(columns = col, data = test)

In [155]:
train.columns

Index(['introelapse', 'testelapse', 'surveyelapse', 'VCL1', 'VCL2', 'VCL3',
       'VCL4', 'VCL5', 'VCL6', 'VCL7',
       ...
       'religion_11.0', 'religion_12.0', 'orientation_1.0', 'orientation_2.0',
       'orientation_3.0', 'orientation_4.0', 'orientation_5.0', 'married_1.0',
       'married_2.0', 'married_3.0'],
      dtype='object', length=298)

In [156]:
# 분류 작업에 필요한 함수 불러오기 
import jinja2
from pycaret.classification import*

# 실험환경 구축 (setup the environment)
- pycaret에서는 모델 학습 전 실험환경을 구축해야한다. Setup 함수를 통해 환경을 구축할 수 있다. 
setup 단계에서는 Pycaret이 자동으로 컬럼 형태를 인식한다. 그후 사용자에게 제대로 인식되었는지 
확인을 받게된다. 그때 enter를 눌러주면 된다. 
또한 주어진 데이터의 얼마를 사용하여 train/validation을 구축할지 묻게되는데, 
전체데이터를 사용하고 싶다면 enter를 눌러주면된다. 

In [157]:
# voted 컬럼이 예측대상이므로 target인자에 명시
clf = setup(data = train, target='nerdiness',fold_shuffle=True)

,Description,Value
0,session_id,5463
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15000, 298)"
5,Missing Values,False
6,Numeric Features,278
7,Categorical Features,19
8,Ordinal Features,False
9,High Cardinality Features,False


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

# 모델 학습 및 비교 
- 환경을 구축했으니 pycaret에서 제공하는 기본 모델에 대해 학습하고 비교해보겠다 
compared_models 함수를 통해 15개의 기본 모델을 학습하고 성능을 비교할 수 있다. 
auc가 기준으로 성능이 가장 좋은 3개의 모델을 추려내어 저장해보겠다. 
본 대회 평가 지표가 auc이기 때문에 auc 기준으로 모델을 선정합니다. 

In [ ]:
best_model

In [158]:
best_3 = compare_models(sort='AUC',n_select=2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7665,0.8560,0.8199,0.7701,0.7941,0.5249,0.5264,0.3100
rf,Random Forest Classifier,0.7661,0.8526,0.8254,0.7668,0.7949,0.5235,0.5256,0.2280
catboost,CatBoost Classifier,0.7539,0.8263,0.8133,0.7569,0.7841,0.4988,0.5006,2.0190
xgboost,Extreme Gradient Boosting,0.7487,0.8190,0.7940,0.7597,0.7764,0.4900,0.4908,4.4720
lightgbm,Light Gradient Boosting Machine,0.7474,0.8183,0.8043,0.7529,0.7777,0.4860,0.4875,3.9920
gbc,Gradient Boosting Classifier,0.7333,0.8014,0.7913,0.7412,0.7653,0.4573,0.4587,0.6030
lr,Logistic Regression,0.7245,0.7970,0.7809,0.7346,0.7570,0.4398,0.4409,0.3840
lda,Linear Discriminant Analysis,0.7240,0.7960,0.7845,0.7324,0.7575,0.4381,0.4396,0.0940
ada,Ada Boost Classifier,0.7205,0.7916,0.7753,0.7320,0.7530,0.4319,0.4329,0.1530
nb,Naive Bayes,0.6761,0.7339,0.7940,0.6748,0.7292,0.3326,0.3401,0.0310


# 모델앙상블
- 3개의 모델을 앙상블 시키도록 한다. 본 대회는 score최적화를 위해 확률값을 예측해야하므로 soft vote ensemble을 진행하겠다.

In [159]:
blended = blend_models(estimator_list = best_3, fold=5, method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7571,0.8411,0.8068,0.7644,0.7850,0.5065,0.5074
1,0.7557,0.8453,0.8180,0.7570,0.7863,0.5022,0.5042
2,0.7790,0.8667,0.8466,0.7729,0.8081,0.5490,0.5522
3,0.7514,0.8438,0.8231,0.7490,0.7843,0.4926,0.4956
4,0.7723,0.8552,0.8378,0.7685,0.8017,0.5355,0.5382
Mean,0.7631,0.8504,0.8265,0.7624,0.7931,0.5171,0.5195
SD,0.0106,0.0094,0.0142,0.0085,0.0099,0.0214,0.0218


# 모델예측
구축된 앙상블모델을 통해 예측을 해보겠습니다. 
setup 환경에 이미 hold-out set이 존재하므로 해당 데이터에 대해 예측을 하여 모델성능을 확인하겠습니다. 

In [122]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7843,0.8679,0.8392,0.7881,0.8129,0.5588,0.5603


# 전체 데이터에 대한 재학습
현재까지 실험은 주어진 train 데이터를 다시 한번 train/validaton으로 나눠서 실험을 한것으로, 전체 train에 대해 학습되어 있지 않습니다. 
최적의 성능을 위해 전체 데이터에 학습을 시켜주도록 하겠다. 

In [160]:
final_model = finalize_model(blended)

In [161]:
predictions = predict_model(final_model, data=test)

In [162]:
predictions

,introelapse,testelapse,surveyelapse,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,...,orientation_1.0,orientation_2.0,orientation_3.0,orientation_4.0,orientation_5.0,married_1.0,married_2.0,married_3.0,Label,Score
0,2.302585,4.744932,5.497168,1,1,1,1,1,1,1,...,0,0,0,1,0,1,0,0,0,0.860
1,2.944439,4.682131,5.332719,1,1,1,1,1,0,0,...,0,0,0,0,1,1,0,0,1,0.710
2,1.609438,4.477337,5.257495,1,1,0,1,1,0,0,...,0,0,0,0,1,1,0,0,1,0.800
3,7.787382,5.049856,5.556828,1,1,1,1,1,1,1,...,0,1,0,0,0,1,0,0,1,0.555
4,3.218876,5.099866,5.669881,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,2.397895,5.303305,5.541264,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0.930
35448,6.224558,4.919981,5.049856,1,0,0,1,1,0,0,...,0,0,0,0,1,1,0,0,1,0.730
35449,3.332205,4.804021,5.257495,1,1,1,1,1,0,0,...,0,0,0,0,1,1,0,0,1,0.925
35450,1.386294,4.077537,4.663439,1,1,0,1,1,1,0,...,1,0,0,0,0,1,0,0,0,0.880


In [163]:
submission['nerdiness'] = predictions['Label']
submission.to_csv('submission_220812_2.csv', index=False)

In [164]:
submission

,index,nerdiness
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,0


In [ ]:
#  Q끼리 pca 한번 해보기 -> 추려진 Q 변수로만 돌려보기 
